In [1]:
# imports
import time
import requests
import pandas as pd
from dotenv import dotenv_values
# constants
url = dotenv_values('.env')['private-rpc']
headers = {"Content-Type": "application/json"}
collection_orderbook = 'F2txTGZbrXdnwR91EETWPU3dK4ZixpW9wmxTEJxFTytm' 
log_list = ['error','Offer','Rescind','Take','Extend','Repay','Foreclose']

In [2]:
# create empty dataframe and lists
schema={'loan':'object','lender':'object','borrower':'object','asset':'object','amount':'object','op_2':'object','op_3':'object','b_time_1':'object','b_time_2':'object','b_time_3':'object','tx_1':'object','tx_2':'object','tx_3':'object'}
df = pd.DataFrame(columns=schema.keys()).astype(schema)
orderbook_tx_list = []
loan_list = []
loan_tx_list = []

In [3]:
# load transactions of collection orderbook into 'orderbook_tx_list' 
payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [collection_orderbook, {"limit": 1000}]}
r = requests.post(url, json=payload, headers=headers)
match r.status_code:
    case 200:   
        data = r.json()
        for x in data['result']:
            if x['confirmationStatus'] != 'processed':
                orderbook_tx_list.append(x['signature'])  
    case _:
        print(r.status_code) 
time.sleep(0.5)          

In [4]:
# ingest offered loans (primary key) into dataframe gathered from transaction details of 'orderbook_tx_list' 
for i in reversed(orderbook_tx_list):
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getTransaction", "params": [i, {"encoding":"json", "maxSupportedTransactionVersion":0}]}
    r = requests.post(url, json=payload, headers=headers)
    match r.status_code:
        case 200:   
            data = r.json()
            logs = []
            if list(data.keys())[1] == 'result':
                for x in log_list:
                    logs.append(len(list(filter(lambda y: x in y, data['result']['meta']['logMessages']))))
                account_keys = data['result']['transaction']['message']['accountKeys']  
                post_blances = data['result']['meta']['postBalances'] 
                instructions = data['result']['transaction']['message']['instructions'] 
                block_time = data['result']['blockTime'] 
                if logs[0]==0 and logs[1]==1:
                    loan = [account_keys[1]]
                    lender = [account_keys[0]]
                    amount = [post_blances[instructions[2]['accounts'][4]]]
                    add_d = pd.DataFrame({'loan':loan,'lender':lender,'amount':amount,'b_time_1':[block_time],'tx_1':[i]})
                    df = pd.concat([df, add_d], ignore_index=True)
                if logs[0]==0 and logs[1]==3:
                    loan = [account_keys[1],account_keys[2],account_keys[3]]
                    lender = [account_keys[0],account_keys[0],account_keys[0]]
                    amount = [post_blances[instructions[2]['accounts'][4]],post_blances[instructions[3]['accounts'][4]],post_blances[instructions[4]['accounts'][4]]]
                    add_d = pd.DataFrame({'loan':loan,'lender':lender,'amount':amount, 'b_time_1':[block_time,block_time,block_time], 'tx_1':[i,i,i]})
                    df = pd.concat([df, add_d], ignore_index=True)
                if logs[1]!=0 and logs[1]!=1 and logs[1]!=3:   
                    print(f'unaccounted loan offer: {i}') 
        case _:
            print(r.status_code) 
    time.sleep(0.5)        

In [5]:
# load transactions of loans into 'loan_tx_list' 
loan_list = list(df['loan']) 
for i in reversed(loan_list):
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [i, {"limit": 10}]}
    r = requests.post(url, json=payload, headers=headers) 
    match r.status_code:
        case 200:   
            data = r.json()
            for x in data['result']:
                if x['confirmationStatus'] != 'processed':
                    loan_tx_list.append(x['signature'])        
        case _:
            print(r.status_code)  
    time.sleep(0.5)  

In [6]:
# ingest loan details into dataframe
for i in loan_tx_list:
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getTransaction", "params": [i, {"encoding":"json", "maxSupportedTransactionVersion":0}]}
    r = requests.post(url, json=payload, headers=headers)
    match r.status_code:
        case 200:   
            data = r.json()
            logs = []
            if list(data.keys())[1] == 'result':
                for x in log_list:
                    logs.append(len(list(filter(lambda y: x in y, data['result']['meta']['logMessages']))))
            account_keys = data['result']['transaction']['message']['accountKeys']
            instructions = data['result']['transaction']['message']['instructions']  
            if logs[0]==0 and logs[2] == 2:
                loan = account_keys[instructions[2]['accounts'][0]]
                df.loc[df.loan == loan, 'op_2'] = 'Rescind' 
                df.loc[df.loan == loan, 'tx_2'] = i 
            if logs[0]==0 and logs[3] == 1:
                loan = account_keys[instructions[2]['accounts'][4]]
                df.loc[df.loan == loan, 'op_2'] = 'Take' 
                df.loc[df.loan == loan, 'borrower'] = account_keys[instructions[2]['accounts'][1]] 
                df.loc[df.loan == loan, 'asset'] = account_keys[instructions[2]['accounts'][2]]
                df.loc[df.loan == loan, 'tx_2'] = i
            if logs[0]==0 and logs[4] == 2: 
                loan_taken = account_keys[instructions[2]['accounts'][1]]
                loan_repaid = account_keys[instructions[2]['accounts'][0]]
                df.loc[df.loan == loan_taken, 'op_2'] = 'Extend' 
                df.loc[df.loan == loan_taken, 'borrower'] = account_keys[instructions[2]['accounts'][2]] 
                df.loc[df.loan == loan_taken, 'asset'] = account_keys[instructions[2]['accounts'][8]]
                df.loc[df.loan == loan_taken, 'tx_2'] = i
                df.loc[df.loan == loan_repaid, 'op_3'] = 'Extend' 
                df.loc[df.loan == loan_repaid, 'tx_3'] = i
            if logs[0]==0 and logs[5] == 2: 
                loan = account_keys[instructions[2]['accounts'][0]]   
                df.loc[df.loan == loan, 'op_3'] = 'Repay'
                df.loc[df.loan == loan, 'tx_3'] = i  
            if logs[0]==0 and logs[6] == 2:  
                loan = account_keys[instructions[2]['accounts'][0]]   
                df.loc[df.loan == loan, 'op_3'] = 'Foreclose'
                df.loc[df.loan == loan, 'tx_3'] = i   
        case _:
            print(r.status_code)         
    time.sleep(0.5)  

In [8]:
df.head()

,loan,lender,borrower,asset,amount,op_2,op_3,b_time_1,b_time_2,b_time_3,tx_1,tx_2,tx_3
0,91Duqu9cLKQcYisPEZ71jUkWhszPAR4R64N7tUTyfmdy,4mt8dNocihMEXoLQguXg567e8x174GZAfV3R5H19VG58,EQEdGHsugMqG3hCmdN5pfgYNsYQMf4WdKdtn884raYzC,9rmfSZkhSGDyPRU8cpjAp8VzVmMuhNWek1kRK3FjivDs,890953520,Take,Extend,1743422859,NaN,NaN,5wUPLchQuvVNesZoWnJKeuqxq1Ybxw6mg1QmCHtNfvnY61...,5Vp4pQd3uS8xdQWxYqaHjT6GZPDtEwsH52bbYxMRS34oEq...,4T7xKocB5dA4xXvMeS3Bzwoinhm3mN2AUBBfSZ1XTgLQoc...
1,4x98rGrGAje4iwKA5PjjASXq5ufVXCgve5Gq7YJVfTCU,H9FDwz926cyVvbv6s4ftmccMBBwaMs9h3k9tavnA8qu8,NaN,NaN,120953520,Rescind,NaN,1743430164,NaN,NaN,4NF68d4kVkrCBxyrHcLRqRZQiQyvYkXjvhaCdnD8xcV1d2...,5WL8JJr8pKKBEnrDctMJRnnp65AjsPtZBD3zjcJ7EXbC9w...,NaN
2,3HDAzi7u7hzNF7PmQ7KVcNW9TnKT9L5WpBVpWePTTZqT,4mt8dNocihMEXoLQguXg567e8x174GZAfV3R5H19VG58,3fAULNcyH95wcCuvKdCuFBcng5PoDzqEWMmpQ4WBWf72,EVzvk9SQWmhF4QAwjLHSWvnyY9QQwTcuYBAQUR6r3EP,400953520,Take,Repay,1743433735,NaN,NaN,2uXNXUd76Zg7MzHDKowAD3B8Rwyao4Z8WMptGr2NL8fGxY...,3J6KffLmN1xT4ZQrNm2UW7eNn67yiWyUvtcBngUA9T5JHn...,5it7shcGmghTC58kZSJfbCvSyaNnEuwKZynfBGt7iH9EPu...
3,5FQAJHHcQdf4YQrBm6SZS4e8JEKcFU9Z3bvE3nwADt4f,4mt8dNocihMEXoLQguXg567e8x174GZAfV3R5H19VG58,NaN,NaN,400953520,Rescind,NaN,1743433735,NaN,NaN,2uXNXUd76Zg7MzHDKowAD3B8Rwyao4Z8WMptGr2NL8fGxY...,4hJV6zzmy13LAANsWa93mtBYHFsY5dXJa26UPMdkMz1ayX...,NaN
4,BAMkVjM9BqBdgiSBvzPGkrpCun9ECBf9kSVunTeyLC23,4mt8dNocihMEXoLQguXg567e8x174GZAfV3R5H19VG58,NaN,NaN,400953520,Rescind,NaN,1743433735,NaN,NaN,2uXNXUd76Zg7MzHDKowAD3B8Rwyao4Z8WMptGr2NL8fGxY...,fW2rP8kyXr8EbMimzEbTyk8e1jf7WidLKCX2CUYxMQSpYz...,NaN


In [10]:
offers = df.query('op_2.isnull()')

In [11]:
offers.loc[:, ['amount']] *= 0.000000001
offers.loc[:, ['amount']] = offers.amount.astype(float).round(2)
offers.loc[:, ['user_readable']]  = offers['lender'].str[0:3] + '___' + offers['lender'].str[-3:]
offers.loc[:, ['datetime']] = pd.to_datetime(offers['b_time_1'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
offers = offers[['amount', 'user_readable', 'datetime']].sort_values(['amount', 'datetime'], ascending=[False, False]).reset_index(drop=True)
offers.index += 1
offers.head(20)

,amount,user_readable,datetime
1,0.4,FRi___FRR,2025-06-03 10:08:02
2,0.39,H9k___P3M,2025-06-03 07:19:15
3,0.39,H9k___P3M,2025-06-03 07:19:15
4,0.39,H9k___P3M,2025-06-03 07:19:15
5,0.32,H9k___P3M,2025-06-06 08:51:36
6,0.31,DGL___tgA,2025-06-05 20:09:47
7,0.3,DGL___tgA,2025-06-02 17:52:45
8,0.28,DGL___tgA,2025-06-01 07:43:03
9,0.1,4XF___eGd,2025-06-01 13:33:05
10,0.05,4XF___eGd,2025-05-07 18:22:52
